In [9]:
import os
import xml.etree.ElementTree as ET
import json
import re

In [10]:
# Set folder path for processing
target = "/mnt/resources/woolworths/raw_data/AustralianSupermarkets.xml"
output_folder = "data/processed_files/xml_output/"

In [16]:
tree = ET.parse(target)
root = tree.getroot()

In [17]:
name=[]
content={}
visited=set()

def explore(root, name: list, content: dict, visited: set):
  if root.tag=="ProcessGroup":
    name.append(root.attrib["Name"])
  for process_group_items in root.findall("ProcessGroupItems"):
    for item in process_group_items:
      if item.tag=="ProcessGroup":
        explore(item, name, content, visited)
        name.pop()
      elif item.tag=="Process":
        section_id=item.attrib["Id"]
        if section_id in visited:
          continue
        visited.add(section_id)
        name.append(item.attrib["Name"])
        key=" --- ".join(name)
        group=item.attrib["Group"]
        page_content=key
        page_content+=f"\n\nName: {item.attrib['Name']}"
        page_content+=f"\nObjective: {item.attrib['Objective']}"
        page_content+=f"\ID: {section_id}"
        page_content+=f"\nGroup: {group}"
        for line in item.iter():
          if line.tag=="Text" or line.tag=="Attachment":
            page_content+="\n"+str(line.text)
        content[key]= (section_id, group, page_content)
        name.pop()
      else:
        print(item.tag)


explore(root, name, content, visited)

In [18]:
len(content)

677

In [44]:
i=0
# d={}
for key, value in content.items():
    filename = re.sub(r"[^\w\s&0-9]|(\s+)", "_", key.lower())
    filename = re.sub(r"_+", "_", filename)
    filename = re.sub(r"(_$)", "", filename)
    filename = "se_"+filename
    filepath = os.path.join(output_folder, filename)
    
    section_id, group, page_content = value
    with open(filepath+".txt", "w") as f:
        f.write(page_content)
    filename, *mid, section_name = key.split(" --- ")

    
    # k=len(key.split(" --- "))
    # if k==6:
    #     print(key)
    # d[k]=d.get(k, 0)+1


    metadata_dict = {
        "group_name":"se",
        "file_type":"section",
        "section_number":section_id,
        "group":group,
        "document_name":mid[0] if len(mid)>0 else "",
        "section_name":section_name,
        "filename":filename,
        "chunk_number":"0",
    }
    with open(filepath+"_metadata.json", "w") as f:
        json.dump(metadata_dict, f)
    i+=1

print(f"Files created: {i}")
    

Files created: 677


In [32]:
len(content)

677

In [33]:
for k, v in sorted(d.items()):
    print(k, v)

2 1
3 43
4 325
5 219
6 66
7 11
8 12


In [20]:
processes=[8628,1995,6939,2398,19092,6939,8706,8616]
output_folder = "data/processed_files/extracted_xml/"


i=0
# d={}
for key, value in content.items():
    filename = re.sub(r"[^\w\s&0-9]|(\s+)", "_", key.lower())
    filename = re.sub(r"_+", "_", filename)
    filename = re.sub(r"(_$)", "", filename)
    filename = "se_"+filename
    filepath = os.path.join(output_folder, filename)
    
    section_id, group, page_content = value
    if int(section_id) in processes:
        with open(filepath+".txt", "w") as f:
            f.write(page_content)
        filename, *mid, section_name = key.split(" --- ")

        
        # k=len(key.split(" --- "))
        # if k==6:
        #     print(key)
        # d[k]=d.get(k, 0)+1


        metadata_dict = {
            "group_name":"se",
            "file_type":"section",
            "section_number":section_id,
            "group":group,
            "document_name":mid[0] if len(mid)>0 else "",
            "section_name":section_name,
            "filename":filename,
            "chunk_number":"0",
        }
        with open(filepath+"_metadata.json", "w") as f:
            json.dump(metadata_dict, f)
        i+=1

print(f"Files created: {i}")

Files created: 7


In [21]:
sorted(processes)

[1995, 2398, 6939, 6939, 8616, 8628, 8706, 19092]